In [1]:


from intensity_normalization.normalize import zscore
import nibabel as nib
import SimpleITK as sitk
import matplotlib.pyplot as plt
import subprocess
import os
import csv
import numpy as np
from intensity_normalization.plot import hist
from intensity_normalization.normalize import fcm
import time
from preprocessing import preprocess
from tqdm import tqdm
import pandas as pd

/home/fabio/.pyenv/versions/3.6.1/lib/python3.6/site-packages/dicom/__init__.py:53: UserWarning: 
This code is using an older version of pydicom, which is no longer 
maintained as of Jan 2017.  You can access the new pydicom features and API 
by installing `pydicom` from PyPI.
See 'Transitioning to pydicom 1.x' section at pydicom.readthedocs.org 
for more information.

  warnings.warn(msg)



1) Bet per creazione maschera. In ogni cartella creo una cartella output contenente la maschera e il cranio che poi elimino
2) N4
3) Ravel per normalizzazione

In [2]:
img_brain = '/media/fabio/Disco locale/Scaricati/ADNI/'

In [3]:
os.chdir('/media/fabio/Disco locale/Scaricati/ADNI/ROBEX/')
print(os.getcwd())

/media/fabio/Disco locale/Scaricati/ADNI/ROBEX


In [6]:
for i, (roots, dirs, files) in enumerate(os.walk(img_brain)):
    if i >=4350:

        if ('Mask' not in roots) and len(files)!=0 and ('ROBEX' not in roots):
            for file in files:
                if '.nii' in file:
                    print(i)
                    os.chdir('/media/fabio/Disco locale/Scaricati/ADNI/ROBEX/')
                    try:
                        os.makedirs(os.getcwd() + '/output/' + '_'.join(file.split('_')[1:4]))
                    except Exception as e:
                        print(e)


                    try:
                        command = ['./ROBEX', roots +'/'+ file,  'output/' + '_'.join(file.split('_')[1:4]) +'/' +file.split('.')[0] +'_robex.nii.gz', 'output/' + '_'.join(file.split('_')[1:4]) + '/'+file.split('.')[0] +'_robex_mask.nii.gz']
                        registration = subprocess.Popen(command, stdout=subprocess.PIPE)
                        output, errors = registration.communicate()
                        print(output, errors)
                    except Exception as e:
                        print(e)

                    os.chdir('/media/fabio/Disco locale/Scaricati/ADNI/')
                    try:
                        command = ['bet', '/'.join(roots.split('/')[6:]) +'/'+ file,  'ROBEX/output/' + '_'.join(file.split('_')[1:4]) +'/' +file.split('.')[0] +'_bet.nii.gz', '-m']
                        registration = subprocess.Popen(command, stdout=subprocess.PIPE)
                        output, errors = registration.communicate()
                        print(output, errors)
                    except Exception as e:
                        print(e)
                    print()

4350
[Errno 17] File exists: '/media/fabio/Disco locale/Scaricati/ADNI/ROBEX/output/037_S_1421'
b'Step 1  of 9: reading in images...Done! It took roughly 1 seconds\nStep 2 of 9: registration...\n  2a) Similarity transform...\n  2b) Affine transform...\n  2c) Resampling volume...\n  Registration done! It took roughly 12 seconds\nStep 3 of 9: rough bias field correction...Done! It took roughly 5 seconds\nStep 4 of 9: calculating features...Done! It took roughly 26 seconds\nStep 5 of 9: voxel classification...Done! It took roughly 12 seconds\nStep 6 of 9: fitting the shape model...Done! It took roughly 2 seconds\nStep 7 of 9: free deformation...Done! It took roughly 1 seconds\nStep 8 of 9: building volume from mesh...Done! It took roughly 2 seconds\nStep 9 of 9: warping back to original space...Done! It took roughly 1 seconds\n\n Output ready \n\n The whole process took roughly 62 seconds\n\n Freeing memory and deleting temporary files...\n\n\n' None
b'' None

4354
[Errno 17] File exists:

b'' None

4386
[Errno 17] File exists: '/media/fabio/Disco locale/Scaricati/ADNI/ROBEX/output/041_S_0679'
b'Step 1  of 9: reading in images...Done! It took roughly 1 seconds\nStep 2 of 9: registration...\n  2a) Similarity transform...\n  2b) Affine transform...\n  2c) Resampling volume...\n  Registration done! It took roughly 10 seconds\nStep 3 of 9: rough bias field correction...Done! It took roughly 5 seconds\nStep 4 of 9: calculating features...Done! It took roughly 22 seconds\nStep 5 of 9: voxel classification...Done! It took roughly 13 seconds\nStep 6 of 9: fitting the shape model...Done! It took roughly 5 seconds\nStep 7 of 9: free deformation...Done! It took roughly 1 seconds\nStep 8 of 9: building volume from mesh...Done! It took roughly 2 seconds\nStep 9 of 9: warping back to original space...Done! It took roughly 0 seconds\n\n Output ready \n\n The whole process took roughly 59 seconds\n\n Freeing memory and deleting temporary files...\n\n\n' None
b'' None

4390
[Errno 17] Fi

In [10]:
MRI_Real = sitk.ReadImage(img_brain)
Brain_MRI = sitk.ReadImage(brain_mask)

mask_brain = sitk.PermuteAxes(Brain_MRI, [2, 1, 0])
flip = sitk.FlipImageFilter()
flip.SetFlipAxes([True, True, True])
flipped_mask = flip.Execute(mask_brain)

RuntimeError: Exception thrown in SimpleITK ReadImage: /tmp/SimpleITK/Code/IO/src/sitkImageReaderBase.cxx:107:
sitk::ERROR: Unable to determine ImageIO reader for "/media/fabio/Disco locale/Scaricati/ADNI/"

In [9]:
img_brain = nib.load(img_brain)
sitk.WriteImage(flipped_mask, brain_mask)
brain_mask = nib.load(brain_mask)
wm_mask = fcm.find_wm_mask(img_brain, brain_mask)
normalized = fcm.fcm_normalize(img_brain, wm_mask)

nib.save(normalized, 'output2.nii.gz')

In [ ]:
ravel.ravel_normalize('/media/fabio/Disco locale/Fabio/010_S_0829/MPR____N3__Scaled/2007-02-08_14_25_09.0/S26117/imgs/', '010_S_0829/MPR____N3__Scaled/2007-02-08_14_25_09.0/S26117/output/', contrast='T1', write_to_disk=True, output_dir='/media/fabio/Disco locale/Scaricati/ADNI_preprocessed_with_skull')

/media/fabio/Disco locale/Scaricati/ROBEX


b'Step 1  of 9: reading in images...Done! It took roughly 0 seconds\nStep 2 of 9: registration...\n  2a) Similarity transform...\n  2b) Affine transform...\n  2c) Resampling volume...\n  Registration done! It took roughly 16 seconds\nStep 3 of 9: rough bias field correction...Done! It took roughly 9 seconds\nStep 4 of 9: calculating features...Done! It took roughly 29 seconds\nStep 5 of 9: voxel classification...Done! It took roughly 16 seconds\nStep 6 of 9: fitting the shape model...Done! It took roughly 6 seconds\nStep 7 of 9: free deformation...Done! It took roughly 1 seconds\nStep 8 of 9: building volume from mesh...Done! It took roughly 3 seconds\nStep 9 of 9: warping back to original space...Done! It took roughly 0 seconds\n\n Output ready \n\n The whole process took roughly 80 seconds\n\n Freeing memory and deleting temporary files...\n\n\n' None


In [22]:
for root, dirs, files in os.walk(img_brain + 'output/'):
    print(files)
    for file in files:
        if 'mask' not in file:
            os.remove(img_brain + 'output/' + file)

['ADNI_002_S_0295_MR_MPR__GradWarp__B1_Correction__N3__Scaled_Br_20070319113623975_S13408_I45108.nii', 'output.nii.gz', 'output_mask.nii.gz']


In [33]:
preprocess('010_S_0829/MPR____N3__Scaled/2007-02-08_14_25_09.0/S26117/', '010_S_0829/MPR____N3__Scaled/2007-02-08_14_25_09.0/S26117/')

In [34]:
ravel.ravel_normalize('010_S_0829/MPR____N3__Scaled/2007-02-08_14_25_09.0/S26117/imgs/', '010_S_0829/MPR____N3__Scaled/2007-02-08_14_25_09.0/S26117/output/', contrast='T1', write_to_disk=True, output_dir='/media/fabio/Disco locale/Scaricati/ADNI_preprocessed_with_skull')

(array([[1.]]), array([[-50.53043747],
        [-50.53043747],
        [-50.53043747],
        ...,
        [-50.53043747],
        [-50.53043747],
        [-50.53043747]]))

In [2]:
lung = len(os.listdir(os.getcwd()))

In [3]:
sorted_list = sorted(os.listdir(os.getcwd()))

In [4]:
sorted_list[1:lung-3]

['002_S_0295',
 '002_S_0413',
 '002_S_0559',
 '002_S_0619',
 '002_S_0685',
 '002_S_0729',
 '002_S_0782',
 '002_S_0816',
 '002_S_0938',
 '002_S_0954',
 '002_S_0955',
 '002_S_1018',
 '002_S_1070',
 '002_S_1155',
 '002_S_1261',
 '002_S_1268',
 '002_S_1280',
 '003_S_0907',
 '003_S_0908',
 '003_S_0931',
 '003_S_0981',
 '003_S_1021',
 '003_S_1057',
 '003_S_1059',
 '003_S_1074',
 '003_S_1122',
 '003_S_1257',
 '005_S_0221',
 '005_S_0222',
 '005_S_0223',
 '005_S_0324',
 '005_S_0448',
 '005_S_0546',
 '005_S_0553',
 '005_S_0572',
 '005_S_0602',
 '005_S_0610',
 '005_S_0814',
 '005_S_0929',
 '005_S_1224',
 '005_S_1341',
 '006_S_0322',
 '006_S_0484',
 '006_S_0498',
 '006_S_0521',
 '006_S_0547',
 '006_S_0653',
 '006_S_0675',
 '006_S_0681',
 '006_S_0731',
 '006_S_1130',
 '007_S_0041',
 '007_S_0068',
 '007_S_0070',
 '007_S_0101',
 '007_S_0128',
 '007_S_0249',
 '007_S_0293',
 '007_S_0316',
 '007_S_0344',
 '007_S_0414',
 '007_S_0698',
 '007_S_1206',
 '007_S_1222',
 '007_S_1248',
 '007_S_1304',
 '007_S_13

In [64]:
for folder in sorted_list[1:lung-3]:
    print("*****")
    print('\n'+folder)
    for root, dirs, files in os.walk(folder):
        if ('Mask' not in root) and len(files)!=0:
            print('\n' + root)
            create_folder(root)

            
            
            

*****

002_S_0295

002_S_0295/MPR__GradWarp__B1_Correction__N3__Scaled/2006-04-18_08_20_30.0/S13408
[Errno 17] File exists: '002_S_0295/MPR__GradWarp__B1_Correction__N3__Scaled/2006-04-18_08_20_30.0/S13408/output/'

002_S_0295/MPR__GradWarp__B1_Correction__N3__Scaled/2006-04-18_08_20_30.0/S13408/output
[Errno 17] File exists: '002_S_0295/MPR__GradWarp__B1_Correction__N3__Scaled/2006-04-18_08_20_30.0/S13408/output/output/'

002_S_0295/MPR__GradWarp__B1_Correction__N3__Scaled_2/2006-04-18_08_20_30.0/S13408
[Errno 17] File exists: '002_S_0295/MPR__GradWarp__B1_Correction__N3__Scaled_2/2006-04-18_08_20_30.0/S13408/output/'
002_S_0295/MPR__GradWarp__B1_Correction__N3__Scaled/2006-04-18_08_20_30.0/S13408/output/
b'' None
002_S_0295/MPR__GradWarp__B1_Correction__N3__Scaled/2006-04-18_08_20_30.0/S13408/output/output/
b'' None
002_S_0295/MPR__GradWarp__B1_Correction__N3__Scaled/2006-04-18_08_20_30.0/S13408/output/output/output/


KeyboardInterrupt: 

In [2]:
df = pd.DataFrame(columns=['ID'])
for i, folder in enumerate(sorted_list[1:lung-3]):
    if i>=104:
        print(i)
        print(folder)            
        for root, dirs, files in os.walk(folder):
            if (not any(word in root for word in ['output', 'imgs', 'preprocessed'])) and (len(files)!=0):
                print(files)
                
                '''

                print(root)
                print("******")
                print("Creazione maschera")

                skull_stripping(root + '/' + files[0], root + '/output/bet_ouput_' + files[0])
                print("Maschera creata\n")

                for root_mask, dirs_mask, files_mask in os.walk(root + '/output'):
                    for file_mask in files_mask:
                        if 'mask' not in file_mask:
                            os.remove(root + '/output/' + file_mask)

                print("Inizio preprocessing")
                preprocess(root, root)
                print("Preprocessing ultimato\n")

                print("Inizio normalizzazione")
                ravel.ravel_normalize(root + '/imgs/', root +'/output/', contrast='T1', write_to_disk=True, output_dir='/media/fabio/Disco locale/Scaricati/ADNI_preprocessed_with_skull')
                print("Terminato \n")

        print(os.listdir(os.getcwd() + '/' + folder))
        all_file = os.listdir(os.getcwd() + '/' + folder)
        mask = [dirs for dirs in all_file if 'Mask' in dirs]
        print(len(mask))
        if len(mask) == 0:
            df = df.append({"ID":folder}, ignore_index=True)
            '''
        
#df.to_csv('missing.csv')

NameError: name 'sorted_list' is not defined

In [6]:
def create_folder(root):
    try:
        os.makedirs(root + '/output/')
    except Exception as e:
        print(e)


In [7]:
try:
    os.makedirs('/media/fabio/Disco locale/Scaricati/ADNI_preprocessed_with_skull')
except Exception as e:
    print(e)

[Errno 17] File exists: '/media/fabio/Disco locale/Scaricati/ADNI_preprocessed_with_skull'


In [8]:
def skull_stripping(img_path, output_path):
    command = ['bet', img_path,  output_path, '-m']
    registration = subprocess.Popen(command, stdout=subprocess.PIPE)
    output, errors = registration.communicate()
    print(output, errors)

In [73]:
root = 'output'

In [84]:
not any(word in root for word in ['Mask', 'ouput'])

True

In [88]:
if not any(word in root for word in ['Mask', 'output']):
    print(word)